In [1]:
import numpy as np
from cmath import *

In [2]:
pres = 10 ** -12

#main LU decomposition logic
def getL(LU):
    dim = LU.shape[0]
    l = LU.copy()
    for k in range(dim):
            l[k, k] = 1
            l[k, k + 1:] = 0
    return l

def getU(LU):
    dim = LU.shape[0]
    u = LU.copy()
    for k in range(1, dim):
        u[k, :k] = 0
    return u

def getLU(a_matrix):
    dim = a_matrix.shape[0]
    a_matrix = a_matrix.copy()
    detSgn = 1
    p_matrix = np.matrix(np.zeros(shape=(dim, dim)))
    q = np.matrix(np.zeros(shape=(dim, dim)))
    np.fill_diagonal(p_matrix, 1)
    np.fill_diagonal(q, 1)
    for k in range(dim - 1): 
        (choseRow, choseCol) = np.unravel_index(np.argmax(np.abs(a_matrix[k:, k:])), a_matrix[k:, k:].shape)
        choseRow += k
        choseCol += k
        if choseRow != k:
            detSgn *= -1
            a_matrix[[choseRow, k]] = a_matrix[[k, choseRow]]
            p_matrix[:, [choseRow, k]] = p_matrix[:, [k, choseRow]]    
        if choseCol != k:
            detSgn *= -1
            a_matrix[:, [choseCol, k]] = a_matrix[:, [k, choseCol]]
            q[[choseCol, k]] = q[[k, choseCol]]
        if np.abs(a_matrix[k, k]) < pres:
            return p_matrix, getL(a_matrix), getU(a_matrix), q, detSgn
        for j in range(k + 1, dim):
            coef = a_matrix[j, k] / a_matrix[k, k]
            a_matrix[j, k:] = a_matrix[j, k:] - a_matrix[k, k:] * coef
            a_matrix[j, k] = coef
    return p_matrix, getL(a_matrix), getU(a_matrix), q, detSgn

#solving
def directSub(l, B, p_matrix):
    dim = p_matrix.shape[0]
    B = p_matrix.transpose() * B
    Y = np.matrix(np.zeros(shape=(dim, 1)))
    for k in range(dim):
        summ = 0
        for j in range(k):
            summ += l[k, j] * Y[j, 0]
        Y[k, 0] = B[k, 0] - summ
    return Y

def inverceSub(u, Y, p_matrix, q):
    dim = p_matrix.shape[0]
    x_ = np.matrix(np.zeros(shape=(dim, 1)))
    for k in range(dim):
        summ = 0
        for j in range(k):
            summ += u[dim - 1 - k, dim - 1 - j] * x_[dim - 1 - j, 0]
        x_[dim - 1 - k, 0] = (Y[dim - 1 - k, 0] - summ) / u[dim - 1 - k, dim - 1 - k]
    return (x_.transpose() * q).transpose()

def solve(a_matrix, b_):
    n = a_matrix.shape[0]
    rangA = getRang(a_matrix)
    if rangA == a_matrix.shape[0]:
        p_matrix, l, u, q, Det = getLU(a_matrix)
        Y = directSub(l, b_, p_matrix)
        x_ = inverceSub(u, Y, p_matrix, q)
        return x_
    if getComp(a_matrix, b_):
        p_matrix, l, u, q, Det = getLU(a_matrix)
        for k in range(rangA, n):
            u[k, k] = 1.
        Y = directSub(l, b_, p_matrix)
        x_ = inverceSub(u, Y, p_matrix, q)
        return x_
    else:
        return 0

#rang & compatibility
def getRang(a_matrix):
    dim = a_matrix.shape[0]
    p, l, u, q, Det = getLU(a_matrix)
    rang = 0
    for k in range(dim):
        if np.abs(u[k, dim - 1]) < pres:
            return rang
        rang += 1
    return rang

def getComp(a_matrix, b_):
    dim = a_matrix.shape[0]
    rangA = getRang(a_matrix)
    AB = np.concatenate((a_matrix, b_), axis=1)
    np.matrix.resize(AB, (dim + 1, dim + 1))
    rangAB = getRang(AB)
    if rangA == rangAB:
        return True
    else:
        return False
    
#solving 3 degree
def cbrt(polynomial):
    solution = set()
    root1 = polynomial ** (1 / 3)
    root2 = (polynomial ** (1 / 3)) * (-1 / 2 + (sqrt(3) * 1j) / 2)
    root3 = (polynomial ** (1 / 3)) * (-1 / 2 - (sqrt(3) * 1j) / 2)
    solution.update({root1, root2, root3})
    return solution

def cardano(a, b, c, d):
    solutions = []
    p = (3 * a * c - b ** 2) / (3 * a ** 2)
    q = (2 * b ** 3 - 9 * a * b * c + 27 * a ** 2 * d) / (27 * a ** 3)
    alpha = cbrt(-q / 2 + sqrt((q / 2) ** 2 + (p / 3) ** 3))
    beta = cbrt(-q / 2 - sqrt((q / 2) ** 2 + (p / 3) ** 3))
    for i in alpha:
        for j in beta:
            if abs((i * j) + p / 3) <= 10**-9:
                x = i + j - b / (3 * a)
                solutions.append(x)
    return solutions

In [19]:
alpha = 0
beta = 3 / 5

# Моя функция
def f(x):
    return 3.7 * np.cos(1.5 * x) * np.exp(-4 * x / 3) + 2.4 * np.sin(4.5 * x) * np.exp(2 * x / 3) + 4

# Дополнительные функции для вычисления mu
def p(x, b_point, beta_value=beta):
        return (b_point - x) ** (-beta_value)

def antiderivative_base(x, b_point, beta_value, degree):
    return ((b_point - x) ** (-beta_value + degree)) / (-beta_value + degree)

old_staff = (
# def antiderivative_p(x, b_point, beta_value=beta):
#         return -antiderivative_base(x, b_point, beta_value, 1)
# 
# def mu_0(a_point, b_point, beta_value=beta):
#         return antiderivative_p(b_point, b_point, beta_value) - antiderivative_p(a_point, b_point, beta_value)
# 
# def antiderivative_x_p(x, b_point, beta_value=beta):
#         return antiderivative_base(x, b_point, beta_value, 2) - b_point * antiderivative_base(x, b_point, beta_value, 1)
# 
# def mu_1(a_point, b_point, beta_value=beta):
#         return antiderivative_x_p(b_point, b_point, beta_value) - antiderivative_x_p(a_point, b_point, beta_value)
# 
# def antiderivative_x2_p(x, b_point, beta_value=beta):
#         return -antiderivative_base(x, b_point, beta_value, 3) + 2 * b_point * antiderivative_base(x, b_point, beta_value, 2) - (b_point ** 2) * antiderivative_base(x, b_point, beta_value, 1)
# 
# def mu_2(a_point, b_point, beta_value=beta):
#         return antiderivative_x2_p(b_point, b_point, beta_value) - antiderivative_x2_p(a_point, b_point, beta_value)
)

def getRow(rowIndex):
    cur_row = []
    cur_row.append(1)
    if rowIndex == 0:
        return cur_row
    prev = getRow(rowIndex - 1)
    for i in range(1, len(prev)):
        curr = prev[i - 1] + prev[i]
        cur_row.append(curr)
    cur_row.append(1)
    return cur_row

def antiderivative_xn_p(x, b_point, beta_value, n):
    pasc_row = getRow(n)
    antiderivative = 0
    for degree in range(1, n + 2):
        mult = pasc_row[degree-1]
        antiderivative += ((-1) ** degree) * mult * (b_point ** (n - degree + 1)) * antiderivative_base(x, b_point, beta_value, degree)
    return antiderivative

def mu_n(a_point, b_point, beta_value, n):
    return antiderivative_xn_p(b_point, 2.3, beta_value, n) - antiderivative_xn_p(a_point, 2.3, beta_value, n)

# Оценка погрешности
def third_derivative(x):
    res = np.exp(-(4 * x) / 3) * (-17.1125 * np.sin(1.5 * x) - 96.4889 * np.exp(2 * x) * np.sin(4.5 * x) + 24.5296 * np.cos(1.5 * x) - 204.3 * np.exp(2 * x) * np.cos(4.5 * x))
    return res

def find_third_derivative_max(space):
    return max(third_derivative(space))

def factorial(n):
    res = 1
    for i in range(1, n + 1):
        res *= i
    return res

def node_polinomal(x, nodes):
    res = 1
    for i in range(len(nodes)):
        res *= x - nodes[i]
    return res

def integrant(x, nodes):
    return abs(p(x, 2.3, beta) * node_polinomal(x, nodes))

def error(a, b):
    nodes = np.linspace(a, b, 3)
    space = np.linspace(a, b, 1000)[0:-1]
    
    res = find_third_derivative_max(space)
    res /= factorial(3)
    res *= space[1] - space[0]
    res *= sum(integrant(space, nodes))
    
    return res

def calculate_step_count(a, b, step_size):
    return int((b - a) / step_size)

def rounge(Sh_2, Sh_1, L, m = 3):
    return (Sh_2 - Sh_1) / (L ** m - 1)

# Интегралл по трём точкам на отрезке (start, stop) от f*p:
def newton_integral_3p(P, fP):
    X = np.matrix((P ** 0, P ** 1, P ** 2))
    MU = np.matrix((mu_n(P[0], P[2], beta, 0), mu_n(P[0], P[2], beta, 1), mu_n(P[0], P[2], beta, 2))).transpose()
    # print(P, '\n', MU, '\n')
    A = solve(X, MU)
    I = np.matrix(fP) * A
    return I

def gaus_integral_3p(P):
    MU_V = np.matrix((mu_n(P[0], P[2], beta, 0), mu_n(P[0], P[2], beta, 1), mu_n(P[0], P[2], beta, 2), mu_n(P[0], P[2], beta, 3), mu_n(P[0], P[2], beta, 4), mu_n(P[0], P[2], beta, 5))).transpose()
    B = -MU_V[3:6]
    MU_matrix = np.matrix(
        (MU_V[0:3].transpose().tolist()[0], MU_V[1:4].transpose().tolist()[0], MU_V[2:5].transpose().tolist()[0]))
    A_V = solve(MU_matrix, B)
    X_S = np.array([x.real for x in cardano(1, A_V[2][0, 0], A_V[1][0, 0], A_V[0][0, 0])])
    X_S.sort()
    X_matrix = np.matrix((X_S ** 0, X_S ** 1, X_S ** 2))
    A = solve(X_matrix, MU_V[0:3])
    return np.matrix(f(X_S)) * A
    

# Составная интегальная формула
def complex_integral(space, space_values, mode='newton'):
    space_l = space[:-1]
    space_r = space[1:]
    
    space_values_l = space_values[:-1]
    space_values_r = space_values[1:]
    
    res = 0
    for i in zip(space_l, space_r, space_values_l, space_values_r):
        middle_point = (i[0] + i[1]) / 2
        local_points = np.array((i[0], middle_point, i[1]))
        if mode == 'newton':
            middle_value = f(middle_point)
            local_values = np.array((i[2], middle_value, i[3]))
            res += newton_integral_3p(local_points, local_values)
        if mode == 'gaus':
            res += gaus_integral_3p(local_points)
            
    return res


def integrate(a, b, mode='newton_3p', num_points=100):
    if mode == 'newton_3p':
        space_3p = np.linspace(a, b, 3)
        return newton_integral_3p(space_3p, f(space_3p))
    
    if mode == 'gaus_3p':
        space_3p = np.linspace(a, b, 3)
        return gaus_integral_3p(space_3p)
    
    if mode == 'gaus_complex':
        space_np = np.linspace(a, b, num_points)
        return complex_integral(space_np, f(space_np), mode='gaus')
    
    if mode == 'newton_complex':
        space_np = np.linspace(a, b, num_points)
        return complex_integral(space_np, f(space_np), mode='newton')
    
    raise ValueError(f'Unknown integration mode: {mode}')


def smart_integrate(a, b, mode='gaus', prescision=0.0001, start_step=0.1, L=2, max_iter=100):
    Sh_1 = np.inf
    Sh_2 = 0
    for i in range(max_iter):
        num_points = calculate_step_count(a, b, start_step)
        if mode == 'gaus':
            Sh_2 = integrate(a, b, mode='gaus_complex', num_points=num_points)
            err = abs(rounge(Sh_2, Sh_1, L, m=6))
        if mode == 'newton':
            Sh_2 = integrate(a, b, mode='newton_complex', num_points=num_points)
            err = abs(rounge(Sh_2, Sh_1, L))
        print(i, start_step, Sh_1, Sh_2, err, abs(integral_value - Sh_2))
        if err < prescision:
            print(f'Presicision {prescision} reached in {i} steps')
            break
        Sh_1 = Sh_2
        start_step /= L
    return Sh_2

## Тесты функций

In [4]:
a = 1.8
b = 2.3
integral_value = 1.185141974956241824914878594317090726677

abs(integral_value - integrate(a, b, mode='newton_complex')[0, 0])

8.186082567362973e-08

In [5]:
error(a, b)

0.983393089410669

In [21]:
abs(integral_value - smart_integrate(a, b, mode='gaus', prescision=10**-8, start_step=0.1, L=2, max_iter=100))

0 0.1 inf [[1.18514284]] [[inf]] [[8.60642216e-07]]
1 0.05 [[1.18514284]] [[1.18514198]] [[1.36284083e-08]] [[2.05249551e-09]]
2 0.025 [[1.18514198]] [[1.18514197]] [[3.25247272e-11]] [[3.43769457e-12]]
Presicision 1e-08 reached in 2 steps


matrix([[3.43769457e-12]])